# Data Preprocessing - Advanced Statistics Topics

In this notebook, we do the general preprocessing and exploratory data analysis of our dataset.

## Import Libraries

In this section we import the requiere libraries and establish the connection with Google Drive.

In [1]:
# Se importan las librerias necesarias
import numpy as np
import pandas as  pd
import os
import gzip
import timeit
# from google.colab import drive # Necesario para GoogleColab

# Tensorflow 2.x
import tensorflow as tf # Not required for this notebook 
print('TENSORFLOW VERSION: {}'.format(tf.__version__))

TENSORFLOW VERSION: 2.4.1


In [2]:
# Se fija el directorio referencia con el se conecta colab a drive # Necesario para GoogleColab
# drive.mount('/content/drive')

In [3]:
# Se guarda en una variable el directorio del curso

# Directorio de los datos
# data_folder = 'drive/MyDrive/Colab Notebooks/AST-ImageCaptioning/Data/'
data_folder = '../Data/'

## Procesamiento de Datos

En esta sección se procesan los datos con los cuales se va a realizar el experimento de: __CPU__ vs __GPU__ vs __TPU__.